# Build Graphs and Networks

This notebook demonstrates how to build various graphs and networks for perturbation analysis.

## Features
- Build co-expression networks
- Create perturbation graphs from GO
- Construct gene-gene interaction networks
- Convert graphs to different formats


In [ ]:
import numpy as np
import pandas as pd
from perturblab.types import GeneVocab
from perturblab.methods.gears import build_perturbation_graph

# Create a gene vocabulary
genes = ['TP53', 'BRCA1', 'KRAS', 'MYC', 'EGFR', 'BRCA2', 'CDKN2A', 'PTEN']
gene_vocab = GeneVocab(genes)
print(f"Gene vocabulary: {len(gene_vocab)} genes")


## Build Perturbation Graph from GO


In [ ]:
# Build perturbation graph using GO annotations
pert_graph = build_perturbation_graph(
    gene_vocab,
    similarity='jaccard',
    threshold=0.1,
    num_workers=1,
    show_progress=True
)

print(f"Perturbation graph:")
print(f"  Nodes: {pert_graph.n_nodes}")
print(f"  Edges: {pert_graph.n_unique_edges}")
print(f"  Average degree: {2 * pert_graph.n_unique_edges / pert_graph.n_nodes:.2f}")

# Query the graph
print(f"\nExample queries:")
for gene in genes[:3]:
    neighbors = pert_graph.neighbors(gene)
    weights = pert_graph.get_weights(gene)
    print(f"  {gene}: {len(neighbors)} neighbors, avg weight: {np.mean(list(weights.values())):.3f}")


## Convert Graph to DataFrame


In [ ]:
from perturblab.methods.gears import weighted_graph_to_dataframe

# Convert to edge list DataFrame
edge_df = weighted_graph_to_dataframe(pert_graph, include_node_names=True)
print(f"Edge DataFrame shape: {edge_df.shape}")
print(f"\nFirst 5 edges:")
print(edge_df.head())

# Can also convert to numeric indices
edge_df_numeric = weighted_graph_to_dataframe(pert_graph, include_node_names=False)
print(f"\nNumeric edge list shape: {edge_df_numeric.shape}")
print(edge_df_numeric.head())
